In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from torchvision.datasets import Food101
from pathlib import Path
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch.utils.data import Subset
from joblib import load, dump

In [2]:
# Creating a directory to save the model
MODEL_DIRECTORY = 'models'
PATH = Path(MODEL_DIRECTORY)
PATH.mkdir(parents= True, exist_ok= True)

MODEL_NAME = 'NaiveBayes.joblib'
MODEL_PATH = PATH / MODEL_NAME

In [3]:
# Transform Layer for images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
    transforms.Lambda(lambda x: x.flatten())  # Flatten the images to a 1D array
])

In [4]:
# Downloading the Food101 Custom Dataset from kaggle.com -> data (directory) 
train_dataset = Food101('data', split= 'train', transform= transform, target_transform= None, download= True)
test_dataset = Food101('data', split= 'test', transform= transform, target_transform= None, download= True)

In [5]:
# Create a subset of the original dataset for training
train_subset = Subset(train_dataset, range(0, 3200))

# Create a subset of the original dataset for validation
val_subset = Subset(test_dataset, range(0, 320))

In [12]:
# Converting Tensors in to Numpy Array
def convert_to_numpy(loader):
    data_list = []
    label_list = []
    for data, labels in tqdm(loader, total=len(loader)):
        data_list.append(data.numpy())
        label_list.append(labels.numpy())
    data_array = np.concatenate(data_list)
    label_array = np.concatenate(label_list)
    return data_array, label_array

In [13]:
# Load the Food101 dataset into PyTorch DataLoader
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

In [14]:
# Convert DataLoader to numpy array
X_train, y_train = convert_to_numpy(train_loader)
X_test, y_test = convert_to_numpy(val_loader)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
# Create a Gaussian Naive Bayes classifier object
model = GaussianNB()

# Fit the classifier on the training data
model.fit(X_train, y_train)

GaussianNB()

In [16]:
# Predict the classes for the validation data
y_pred = model.predict(X_test)

In [17]:
# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 34.69%


In [18]:
# Saving the model to desired path
print(f'Saving model to: {MODEL_PATH}')
dump(model, MODEL_PATH)

Saving model to: models\NaiveBayes.joblib


['models\\NaiveBayes.joblib']

In [19]:
# Loading the model with saved model weights
model_0 = load(MODEL_PATH)